In [ ]:
# This notebook uses the VGG16 neural network to create features for
# finding similar images. This has been rewritten for Pytorch.
# VGG16 was trained in ImageNet and can provide object detection, which can
# be useful with these images and the network's representation of scaled images
# as a vector of 1,000 values provides useful features for distance measurements,
# classifications, and other image tasks. 
#
# James E. Dobson
# James.E.Dobson@Dartmouth.EDU
# https://jeddobson.github.io/

import numpy as np
from glob import glob
import cv2
from matplotlib import pyplot as plt
from PIL import Image
import numpy, os, re
import tensorflow as tf
import torch, torchvision
from torchvision.models import vgg16
from torchvision.io import read_image
import sklearn

In [ ]:
model = vgg16(weights='DEFAULT')
weights = torchvision.models.VGG16_Weights.DEFAULT
preprocess = weights.transforms()

# put model into eval state
model.eval()

In [ ]:
def display_image(image_file):
    img = np.asarray(Image.open(image_file))
    plt.imshow(img)

In [ ]:
def decode_predictions(prediction,topk=5):
    # can't figure out how to make tensor's argsort reverse like this
    class_ids = np.argsort(prediction.detach().numpy())[::-1][:topk]
    for i in class_ids:
        score = prediction[i].item()
        category_name = weights.meta["categories"][i]
        print(f"{category_name}: {100 * score:.1f}%")

In [ ]:
def get_prediction(image_file,display_flag=False):
    img = read_image(image_file)
    batch = preprocess(img).unsqueeze(0)
    prediction = model(batch).squeeze(0).softmax(0)
    if display_flag:
        display_image(image_file)
    decode_predictions(prediction)

In [ ]:
get_prediction('train/14719691_657210427789980_5511428924740993024_n.jpg',display_flag=True)

In [ ]:
get_prediction('train/14717662_219665258446439_2538176967283310592_n.jpg',display_flag=True)

In [ ]:
get_prediction('train/14482663_1739187826342894_146245723782905856_n.jpg',display_flag=True)

In [ ]:
get_prediction('train/14566625_1760095214239415_7171580328929132544_n.jpg',display_flag=True)

In [ ]:
dartmouth_instagram = list()
files = glob("data/*.jpg")[:25]
for file in files:
    img = read_image(file)
    batch = preprocess(img).unsqueeze(0)
    prediction = model(batch).squeeze(0).softmax(0)
    dartmouth_instagram.append(prediction.detach().numpy())

In [ ]:
from sklearn.neighbors import KDTree
kdt = KDTree(dartmouth_instagram, leaf_size=30, metric='euclidean')
neighbors = kdt.query(dartmouth_instagram, k=10, return_distance=False)    

In [ ]:
def display_neighbors(neighboring_images):
    plt.figure(figsize=(15,7), facecolor='white')
    i = 1
    for image in neighboring_images:
        ax = plt.subplot(2, 5, i)
        
        # read and resize image to 200 x 200
        img = cv2.imread(files[image])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (200, 200)) 
        ax.axis('off')
        ax.imshow(img)
        i += 1
    plt.tight_layout()
    plt.show()

In [ ]:
display_neighbors(neighbors[0])

In [ ]:
# calculate similarities with Euclidean distance metric
from sklearn.metrics import euclidean_distances
euclidean_dist_matrix = euclidean_distances(dartmouth_instagram)

In [ ]:
for i in np.argsort(euclidean_dist_matrix[16]):
    print(files[i],euclidean_dist_matrix[16][i])
    img = cv2.imread(files[i])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.show()